Treinando uma rede neural usando pytorch
========================================



## Introdução



O notebook anterior foi o *season finale* da construção da rede neural usando Python puro. Este notebook é o epílogo.

Aqui veremos como podemos construir e treinar uma rede neural usando `pytorch`, um módulo especializado em redes neurais artificiais.

Motivos para usarmos o `pytorch`:

1.  A `MLP` que criamos em Python puro não é otimizada. Tente treinar uma rede com uns 100 neurônios em uma camada e verá como ela é bem lenta! `pytorch` é escrito por diversos programadores que além de primar pelas contas estarem corretas, eles fazem de tudo para que elas ocorram de forma eficiente.

2.  `pytorch` já tem praticamente tudo que precisamos implementado! Quer calcular o gradiente local da função arco tangente? Ele já tem! Quer usar a função de ativação ReLU? Ele já tem!

3.  `pytorch` tem suporte a treinar uma rede neural usando processamento gráfico (GPU). Isso acelera muito o treino de redes neurais artificiais complexas. Temos GPUs para usarmos no HPC da Ilum!

4.  `pytorch` é usado tanto na academia quanto no mundo corporativo. Junto com `tensorflow` são os dois módulos estado da arte para o projeto e treino de redes neurais.



## Objetivo



Treinar uma rede neural artificial tipo Multilayer Perceptron usando `pytorch`.



## Importações



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

c:\venv\ilumpy\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Código e discussão



### Dados



Vamos treinar nossa rede neural com nosso velho amigo, o dataset de diamantes! Como todo processo de treino de aprendizado de máquina, precisamos reservar um conjunto de dados para treino e outro para teste.



In [2]:
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 61455
DATASET_NAME = "diamonds"
FEATURES = ["carat", "depth", "table", "x", "y", "z"]
TARGET = ["price"]

df = sns.load_dataset(DATASET_NAME)

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

X_treino = df_treino.reindex(FEATURES, axis=1)
y_treino = df_treino.reindex(TARGET, axis=1)
X_teste = df_teste.reindex(FEATURES, axis=1)
y_teste = df_teste.reindex(TARGET, axis=1)

Redes neurais costumam se dar bem quando os dados estão entre $[0, 1]$ ou entre $[-1, 1]$. Redes neurais costumam não se dar bem quando os dados estão em escalas muito diferentes. Por conta disso, normalizar os dados antes de prosseguir é recomendado! Vamos usar o `MinMaxScaler` do `scikit-learn` que aprendemos semestre passado.



In [3]:
normalizador_x = MinMaxScaler()
normalizador_y = MinMaxScaler()

normalizador_x.fit(X_treino)
normalizador_y.fit(y_treino)

X_treino = normalizador_x.transform(X_treino)
y_treino = normalizador_y.transform(y_treino)
X_teste = normalizador_x.transform(X_teste)
y_teste = normalizador_y.transform(y_teste)

### Tensores



Na nossa rede neural feita em Python puro, a base de tudo era a classe `Valor`. Para o `pytorch`, a base de tudo são os tensores! Tensores são objetos que armazenam dados de forma similar aos arrays de `numpy`, porém registram todas as operações realizadas para o cálculo do gradiente local (assim como nós fizemos com a classe `Valor`).



In [4]:
X_treino = torch.tensor(X_treino, dtype=torch.float32)
y_treino = torch.tensor(y_treino, dtype=torch.float32)
X_teste = torch.tensor(X_teste, dtype=torch.float32)
y_teste = torch.tensor(y_teste, dtype=torch.float32)

Vamos checar os dados para ver como estão.



In [5]:
print(X_treino)
print()
print(y_treino)

tensor([[0.0686, 0.5611, 0.2885, 0.4777, 0.0876, 0.1022],
        [0.0624, 0.5250, 0.2500, 0.4730, 0.0871, 0.0994],
        [0.0478, 0.4917, 0.2692, 0.4572, 0.0829, 0.0934],
        ...,
        [0.1185, 0.5333, 0.2692, 0.5410, 0.0995, 0.1142],
        [0.0166, 0.5139, 0.2885, 0.3929, 0.0708, 0.0811],
        [0.0333, 0.5361, 0.2692, 0.4209, 0.0776, 0.0890]])

tensor([[0.0676],
        [0.0586],
        [0.0434],
        ...,
        [0.1527],
        [0.0172],
        [0.0216]])


### Criando a rede neural



Para criar uma rede neural usando `pytorch` é necessário criar uma classe. Observe aqui que a classe criada é baseada na classe `nn.Module` do `pytorch`. Classes podem ser baseadas em outras classes! Trata-se da característica de herança das classes.

Observe que definimos as camadas da rede neural dentro de um objeto `nn.Sequential`.

Observe também o método `forward`. Este funciona de forma similar ao dunder `__call__` que vimos anteriormente. O `pytorch` requer que usemos o `forward` e não o `__call__`, então aceitamos e seguimos em frente.



In [6]:
class MLP(nn.Module):
    def __init__(
        self, num_dados_entrada, neuronios_c1, neuronios_c2, num_targets
    ):
        # Temos que inicializar a classe mãe
        super().__init__()

        # Definindo as camadas da rede
        self.camadas = nn.Sequential(
                    nn.Linear(num_dados_entrada, neuronios_c1),
                    nn.ReLU(),
                    nn.Linear(neuronios_c1, neuronios_c2),
                    nn.ReLU(),
                    nn.Linear(neuronios_c2, num_targets),
                )

    def forward(self, x):
        """Esse é o método que executa a rede do pytorch."""
        x = self.camadas(x)
        return x

Agora podemos criar a nossa rede neural!



In [7]:
NUM_DADOS_DE_ENTRADA = X_treino.shape[1]
NUM_DADOS_DE_SAIDA = y_treino.shape[1]
NEURONIOS_C1 = 50
NEURONIOS_C2 = 20

minha_MLP = MLP(NUM_DADOS_DE_ENTRADA, NEURONIOS_C1, NEURONIOS_C2, NUM_DADOS_DE_SAIDA)

E podemos checar os parâmetros internos dela!



In [8]:
for p in minha_MLP.parameters():
    print(p)

Parameter containing:
tensor([[-0.0140, -0.0283,  0.2253,  0.1262, -0.2799,  0.2314],
        [-0.0676,  0.1150, -0.3239,  0.3064, -0.0710, -0.1175],
        [ 0.3124,  0.1934, -0.0649,  0.1261,  0.1928, -0.1205],
        [ 0.0602,  0.0562,  0.1754, -0.2576,  0.0349,  0.3436],
        [-0.0208, -0.0060,  0.2893,  0.2180,  0.0443, -0.0949],
        [ 0.3220,  0.0798,  0.1410, -0.0558, -0.1988, -0.2931],
        [-0.1444, -0.1000,  0.3891, -0.3483,  0.0777,  0.4036],
        [-0.2996,  0.0044,  0.4016, -0.1807, -0.2669,  0.3132],
        [-0.1419, -0.1520, -0.0053,  0.2540, -0.3391, -0.0097],
        [-0.2506,  0.0202,  0.1277, -0.0533,  0.1182, -0.1728],
        [ 0.0910, -0.0980,  0.1116, -0.0241, -0.0639, -0.0730],
        [-0.0205, -0.1600,  0.3279,  0.2244, -0.2857, -0.0417],
        [-0.3289,  0.0779,  0.2220, -0.1697,  0.3427, -0.1739],
        [ 0.1409,  0.1407,  0.1561,  0.3098,  0.3495, -0.4021],
        [ 0.3426,  0.1388,  0.1630, -0.3610,  0.1742,  0.2506],
        [ 0.3233, 

Também podemos realizar uma previsão, mas ela provavelmente será bem ruim!



In [9]:
y_prev = minha_MLP(X_treino)
y_prev

tensor([[0.3352],
        [0.3332],
        [0.3293],
        ...,
        [0.3386],
        [0.3243],
        [0.3287]], grad_fn=<AddmmBackward>)

### A função de perda e o otimizador



O `pytorch` já vem com diversas funções de perda já implementadas. A que computa o erro quadrático médio, por exemplo, já está pronta!

Fora isso, precisamos definir o nosso otimizador. O otimizador é quem cuida de atualizar os parâmetros da nossa rede. Nós implementamos na nossa rede em Python puro a descida do gradiente. Aqui usaremos um otimizador que é uma modificação da descida do gradiente, chamado `Adam`. Trata-se, simplificadamente, de uma descida do gradiente com taxa de aprendizado individualizada para cada parâmetro e que se altera ao longo do aprendizado. `Adam` é um poderosíssimo otimizador! Não tem porque não usarmos ele neste caso.



In [10]:
TAXA_DE_APRENDIZADO = 0.001

# função perda será o erro quadrático médio
fn_perda = nn.MSELoss()

# otimizador será o Adam, um tipo de descida do gradiente
otimizador = optim.Adam(minha_MLP.parameters(), lr=TAXA_DE_APRENDIZADO)

### O treino da rede



Para avisar o `pytorch` que iremos treinar a rede, vamos invocar o método `train` da nossa rede. Toda rede recém-criada já está no modo treino, mas aqui deixarei explícito para ficar bem claro.



In [11]:
minha_MLP.train()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

Observe que o ciclo do treino tem as mesmas etapas que vimos no notebook anterior, nada de novo! Aqui basta observar como executamos essas etapas no `pytorch`.



In [ ]:
NUM_EPOCAS = 1000

y_true = y_treino

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = minha_MLP(X_treino)

    # zero grad
    otimizador.zero_grad()

    # loss
    loss = fn_perda(y_pred, y_true)

    # backpropagation
    loss.backward()

    # atualiza parâmetros
    otimizador.step()

    # mostra resultado
    print(epoca, loss.data)

0 tensor(0.0638)
1 tensor(0.0606)
2 tensor(0.0578)
3 tensor(0.0553)
4 tensor(0.0531)
5 tensor(0.0511)
6 tensor(0.0491)
7 tensor(0.0474)
8 tensor(0.0462)
9 tensor(0.0454)
10 tensor(0.0447)
11 tensor(0.0442)
12 tensor(0.0437)
13 tensor(0.0434)
14 tensor(0.0431)
15 tensor(0.0428)
16 tensor(0.0426)
17 tensor(0.0424)
18 tensor(0.0422)
19 tensor(0.0421)
20 tensor(0.0418)
21 tensor(0.0415)
22 tensor(0.0412)
23 tensor(0.0409)
24 tensor(0.0405)
25 tensor(0.0400)
26 tensor(0.0396)
27 tensor(0.0392)
28 tensor(0.0387)
29 tensor(0.0383)
30 tensor(0.0379)
31 tensor(0.0375)
32 tensor(0.0371)
33 tensor(0.0367)
34 tensor(0.0364)
35 tensor(0.0360)
36 tensor(0.0355)
37 tensor(0.0351)
38 tensor(0.0345)
39 tensor(0.0339)
40 tensor(0.0333)
41 tensor(0.0328)
42 tensor(0.0322)
43 tensor(0.0316)
44 tensor(0.0311)
45 tensor(0.0305)
46 tensor(0.0298)
47 tensor(0.0292)
48 tensor(0.0286)
49 tensor(0.0279)
50 tensor(0.0272)
51 tensor(0.0266)
52 tensor(0.0259)
53 tensor(0.0252)
54 tensor(0.0245)
55 tensor(0.0239)
56

Após o treino, podemos checar a performance da nossa rede. Observe a linha que inicia com `with`. Tudo que estiver no bloco do `with torch.no_grad()` é computado normalmente, porém o grafo computacional e os gradientes locais não são computados. Use isso sempre que for realizar contas com tensores fora do treino!!



In [ ]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_treino)
    y_pred = minha_MLP(X_treino)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

### O teste da rede



Vamos supor que você já testou diversas arquiteturas de rede e está pronto para usar os dados de teste. Nesta etapa precisamos indicar para o `pytorch` que o treino acabou e estamos na etapa de avaliação. Fazemos isso rodando o método `eval`.



In [ ]:
minha_MLP.eval()

Agora podemos realizar o teste!



In [ ]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_teste)
    y_pred = minha_MLP(X_teste)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

E, finalmente, computar alguma métrica para medir a performance do nosso modelo.



In [ ]:
RMSE = mean_squared_error(y_true, y_pred, squared=False)
print(f'Loss do teste: {RMSE}')

## Conclusão

<p style="text-align: justify"> Para aqueles que acreditam no final da jornada, era pegadinha! O encerramento oficial do guia de redes neurais se encerra no notebook oito, onde apresentamos um spin off do treinamento de uma rede neural! Esse spin off ocorre porque utilizamos um módulo especializado para redes neurais artificiais denominada de Pytorch.  </p>
<p style="text-align: justify"> De modo a demonstrar o funcionamento desse módulo, foi utilizada a biblioteca de diamantes em que o target é preço desses diamantes e os atributos que vão nos ajudar a predizer esse preço são "carat", "depth", "table", "x", "y" e "z".  O dataframe é dividio em modelo e este, para enfim, ser normalizado entre [0,1] ou [-1,1], porque os dados não são tão bem trabalhados quando há escalas diferentes. </p>
<p style="text-align: justify"> Uma propriedade interessante do PyTorch é que diferente do Python Puro, ele não utiliza as classes como base e sim uma propriedade denominada de Tensores, sendo responsáveis por representar os dados de entrada, os pesos e os gradientes das redes neurais, podendo ter diferentes dimensões que vão de 0 (tensor escalar) a n (tensor de ordem n). Outra vantagem é que esse módulo possui diversas funções de perdas já implementadas, como a que computa o erro quadrático médio, só que para isso ocorrer devemos definir o nosso otimizador, o responsável pela atualização dos parâmetros da nossa rede, anteriormente o que viamos com o nome de descida do gradiante passa a ser substituído pelo otimizador Adam que essencialmente, diz respeito a uma descida do gradiente com taxa de aprendizado individualizada para cada parâmetro que sofre com mudanças ao longo do aprendizado.</p>
<p style="text-align: justify"> O treinamento da rede ocorre pelo método train que excecuta todo o processo anterior de treinamento por poucas linhas de código. Após esse treino, realiza-se o método RMSE para análise do modelo de rede.</p>


## Playground



In [ ]:
print("Muito obrigada pela aula, Dani :)")